In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [7]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')
f.make_table_features_process(stock_set, interval, 8)
f.make_master_table(stock_set, interval)
f.xg_boost_model(interval)

Stock set: 
{'MA', 'VZ', 'XEL', 'GS', 'AJG', 'MRK', 'AMZN', '^VIX', 'GOOGL', 'NRG', 'TTWO', 'KO', 'SRE', 'IBM', 'CVX', 'COST', 'WSM', 'WMT', 'CEG', 'DUK', 'D', 'PSX', 'ATO', 'NFLX', 'TMUS', 'SLB', 'RCL', 'UNH', 'FWONK', 'DASH', 'PGR', 'ABT', 'GOOG', 'PG', 'EOG', 'EXC', 'NEE', 'LLY', 'TDG', 'SO', 'ABBV', 'VST', 'MDLZ', 'CMCSA', 'SW', 'BAC', 'ACN', 'WMB', 'XOM', 'BRK-B', 'APH', 'LNG', 'HD', 'DIS', 'ORCL', 'FCNCA', 'PEP', 'MSI', 'AAPL', 'CSCO', 'DECK', 'JNJ', 'T', 'ADBE', 'WELL', 'MO', 'JPM', 'TSLA', 'WFC', 'TMO', 'NVDA', 'CL', 'AVGO', 'EXE', 'PM', 'CRM', 'COP', 'KMI', 'TGT', 'TPR', 'ISRG', 'DKNG', 'AEP', 'META', 'MSFT', 'SPGI', 'AMGN', 'OKE', 'BSX', 'V', 'PLTR', 'AXP'}

Saved combined dataframe to: ./data_transformed/all_1d_model_df.pkl


/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [23:41:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.38      0.37      0.38     55460
           1       0.41      0.40      0.41     55385
           2       0.41      0.43      0.42     55417

    accuracy                           0.40    166262
   macro avg       0.40      0.40      0.40    166262
weighted avg       0.40      0.40      0.40    166262


Feature Importances (Descending):
                    feature  importance
38     cat__day_of_month_23    0.030315
19      cat__day_of_month_4    0.026841
39     cat__day_of_month_24    0.026312
46     cat__day_of_month_31    0.025483
42     cat__day_of_month_27    0.025298
13    cat__month_of_year_10    0.023737
6      cat__month_of_year_3    0.023681
33     cat__day_of_month_18    0.022973
11     cat__month_of_year_8    0.022892
27     cat__day_of_month_12    0.022491
15    cat__month_of_year_12    0.022368
23      cat__day_of_month_8    0.022214
28     cat__day_of_month_13    0.022187
25     cat__day_of_month

In [8]:
f.model_prospect('nvda', '1d')


Model Prediction 1d NVDA:
Predicted Next 1d Movement: UP

Model Prediction Probabilities:
no_change (0): 0.3285
up (1): 0.3846
down (2): 0.2869

Last Entry 1d NVDA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-01-15 12:00:00 AM EST-0500
PDT: 2025-01-14 09:00:00 PM PST-0800
